In [135]:
import requests as r
import pandas as pd
import numpy as np

### Se construirá un dataset de muestra de los sellers de todas las categorías disponibles en Mercado Libre Argentina
Para ello se tomaron aplicará la siguiente metodología:
1. El listado de las categorías, con su correspondiente id. Se trabajará con 10 categorias elegidas aleatoreamente
2. Se buscarán los primeros 200 items de dichas categorias y se extraerán los datos de los sellers que se consideran pertinentes para el análisis, 
    esto nos permitirá obtener una muestra de sellers de distintos rubros y categorias, se buscó bajar la influencia de las publicaciones más relevantes
    y al quitar duplicados de sellers, se tendrá un listado de candidatos para **identificar a aquellos que tienen un buen perfil y son relevantes para el negocio**
3. Dado que en los datos de los sellers dentro de cada item, tiene datos referidos:
    - provincia y localidad
    - Cancelaciones, Reclamos y Demoras de los últimos 60 días (cantidad y porcentaje del total de operaciones)
    - El total de transacciones históricas: canceladas y completas
    - Rating de los usuarios: negativo, neutral y positivos
    Estas variables se consideran valiosas y necesarias para el análisis
4. Es relevante para el negovio contar con información acerca de la facturación de los sellers: Por lo tanto, de los sellers obtenidos en el paso anterior 
    se buscarán todos los items del seller, para obtener el listado de productos, sus precios y cantidades vendidas,
    con esto se espera obtener una metrica de facturacion y una metrica aproximada de la ganancia del negocio por cada seller. Luego se agruparán por seller los resultados 
    para continuar trabajando con dicha unidad de análisis. 
5. Se unirán los datos por USER_ID y está será la muestra de datos con el que se continuará el análisis

Estás decisiones se basan en la hipotesis de que los sellers con mejores metricas, ratings, transacciones y rentabilidad serán los relevantes y tengan un buen perfil para el negocio
Respecto de la construccion del dataset, se indagó en la documentación de la API y en sus resultados y se optó por esta combinacion de consultas para llegar al dataset final


## CREACION DEL DF CATEGORIAS
### Contiene los datos de todas las categorias disponibles en Mercado Libre Argentina

In [158]:
url = "https://api.mercadolibre.com/sites/MLA/categories"
request = r.get(url)
results = request.json()
categorias = pd.DataFrame(results)
categorias = categorias.sample(10).reset_index(drop=True)

In [159]:
categorias

,id,name
0,MLA5726,Electrodomésticos y Aires Ac.
1,MLA1168,"Música, Películas y Series"
2,MLA1648,Computación
3,MLA1071,Animales y Mascotas
4,MLA1384,Bebés
5,MLA3025,"Libros, Revistas y Comics"
6,MLA1743,"Autos, Motos y Otros"
7,MLA1500,Construcción
8,MLA1276,Deportes y Fitness
9,MLA1132,Juegos y Juguetes


In [166]:
# se extraen las categorias para mantener la muestra
categorias.to_csv('../data/muestra_categorias.csv')

In [ ]:
categorias = pd.read_csv('../data/muestra_categorias.csv')

In [167]:
offset_list = np.arange(0, 1000, 50).tolist()

In [168]:
df_items = pd.DataFrame()
idx = 0

for idx in range(len(categorias)):

    categoria = categorias.id[idx]

    for offset in offset_list:     

        url = f'https://api.mercadolibre.com/sites/MLA/search?category={categoria}&offset={offset}'

        req = r.get(url)

        items = req.json()['results']

        for item in range(len(items)):

            aux = pd.DataFrame({
                "categoria_MLA" : categoria,
                "seller" : items[item]['seller']['id'],
                "fecha_registro" : items[item]['seller']['registration_date'],
                "power_seller_status" :items[item]['seller']['seller_reputation']['power_seller_status'],
                "level_id" :items[item]['seller']['seller_reputation']['level_id'],
                "metrics_cancellations_value" :items[item]['seller']['seller_reputation']['metrics']['cancellations']['value'],
                "metrics_cancellations_rate" :items[item]['seller']['seller_reputation']['metrics']['cancellations']['rate'],
                "metrics_claims_value" :items[item]['seller']['seller_reputation']['metrics']['claims']['value'],
                "metrics_claims_rate" :items[item]['seller']['seller_reputation']['metrics']['claims']['rate'],
                "metrics_delayed_handling_time_value" :items[item]['seller']['seller_reputation']['metrics']['delayed_handling_time']['value'],
                "metrics_delayed_handling_time_rate" :items[item]['seller']['seller_reputation']['metrics']['delayed_handling_time']['rate'],
                "metrics_sales_value" :items[item]['seller']['seller_reputation']['metrics']['sales']['completed'],
                "transactions_historic_total": items[item]['seller']['seller_reputation']['transactions']['total'],
                "transactions_historic_canceled": items[item]['seller']['seller_reputation']['transactions']['canceled'],
                "transactions_historic_completed": items[item]['seller']['seller_reputation']['transactions']['completed'],
                "rating_negative": items[item]['seller']['seller_reputation']['transactions']['ratings']['negative'],
                "rating_neutral": items[item]['seller']['seller_reputation']['transactions']['ratings']['neutral'],
                "rating_positive": items[item]['seller']['seller_reputation']['transactions']['ratings']['positive'],

            }, index = {idx})
            df_items = df_items.append(aux)
            idx += 1

In [ ]:
df_items.value_counts('categoria_MLA')

categoria_MLA
MLA1168    1000
MLA1743    1000
MLA1384     999
MLA1071     998
MLA1648     996
MLA5726     995
MLA3025     962
MLA1500      47
dtype: int64

In [ ]:
df_items.sample(10)

,categoria_MLA,seller,fecha_registro,power_seller_status,level_id,metrics_cancellations_value,metrics_cancellations_rate,metrics_claims_value,metrics_claims_rate,metrics_delayed_handling_time_value,metrics_delayed_handling_time_rate,metrics_sales_value,transactions_historic_total,transactions_historic_canceled,transactions_historic_completed,rating_negative,rating_neutral,rating_positive
0,MLA5726,608846165,2020-07-13T13:21:30.000-04:00,platinum,5_green,0,0.0000,788,0.0071,0,0.0000,104352,814242,41242,773000,0.05,0.02,0.93
1,MLA5726,4641264,2001-06-19T00:00:00.000-04:00,platinum,5_green,373,0.0067,754,0.0136,197,0.0036,52733,289721,15953,273768,0.04,0.02,0.94
2,MLA5726,132422133,2013-01-24T16:59:01.000-04:00,platinum,5_green,0,0.0000,18,0.0090,0,0.0000,1900,16569,927,15642,0.06,0.05,0.89
3,MLA5726,4641264,2001-06-19T00:00:00.000-04:00,platinum,5_green,373,0.0067,754,0.0136,197,0.0036,52733,289721,15953,273768,0.04,0.02,0.94
4,MLA5726,15604260,2011-12-21T00:19:37.000-04:00,platinum,5_green,9,0.0026,48,0.0141,69,0.0219,3178,25338,1694,23644,0.04,0.03,0.93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,MLA1500,313499472,2018-04-09T11:56:55.000-04:00,gold,5_green,3,0.0098,3,0.0098,2,0.0134,296,2252,110,2142,0.01,0.02,0.97
50,MLA1500,132453365,2013-01-25T10:23:05.000-04:00,platinum,5_green,170,0.0118,123,0.0085,164,0.0203,13548,78865,4641,74224,0.02,0.02,0.96
51,MLA1500,171245002,2014-11-19T10:26:01.000-04:00,platinum,5_green,11,0.0020,70,0.0130,100,0.0221,5055,24507,1482,23025,0.02,0.02,0.96
52,MLA1500,139673546,2013-06-03T18:54:12.000-04:00,platinum,5_green,70,0.0037,246,0.0131,69,0.0040,17711,131820,7393,124427,0.02,0.02,0.96


In [ ]:
df_items.to_parquet('muestraSellers_dataMLA.parquet')